In [1]:
import glob, dispindiffs
import polars as pl
import pandas as pd
import numpy as np

## Airport information

In [2]:
ourairports = pl.read_csv("./dat/airports.csv") # https://ourairports.com/data/ - "airports.csv"
iata_to_local = dict(zip(ourairports["iata_code"], ourairports["local_code"]))

airport_info = pl.read_excel("./dat/ARP-NPIAS-2025-2029-AppendixA.xlsx") # https://www.faa.gov/airports/planning_capacity/npias/current - Appendix A xlsx file
airport_info = airport_info.with_columns(pl.col("Role\r\n(FY25)").fill_null(""))

In [3]:
# https://www.faa.gov/airports/planning_capacity/npias/current
region_to_states = {
    "Alaskan": ["Alaska"],
    "Central": ["Nebraska", "Iowa", "Kansas", "Missouri"],
    "Eastern": ["Delaware", "Maryland", "New Jersey", "New York", "Pennsylvania", "Virginia", "West Virginia"],
    "Great Lakes": ["Illinois", "Indiana", "Michigan", "Minnesota", "North Dakota", "Ohio", "South Dakota", "Wisconsin"],
    "New England": ["Connecticut", "Maine", "Massachusetts", "New Hampshire", "Rhode Island", "Vermont"],
    "Northwest Mountain": ["Colorado", "Idaho", "Montana", "Oregon", "Utah", "Washington", "Wyoming"],
    "Southern": ["Alabama", "Florida", "Georgia", "Kentucky", "Mississippi", "North Carolina", "Puerto Rico", 
                 "South Carolina", "Tennessee", "Virgin Islands"],
    "Southwest": ["Arkansas", "Louisiana", "New Mexico", "Oklahoma", "Texas"],
    "Western-Pacific": ["American Samoa", "Arizona", "California", "Guam", "Hawaii", "Nevada", "Pacific Islands"]
}
state_to_region = {}
for r in region_to_states:
    for s in region_to_states[r]:
        state_to_region[s] = r
code_to_region = {}
for row in airport_info.iter_rows(named=True):
    code_to_region[row["LocID"]] = state_to_region[row["State"]]

code_to_hub = dict(zip(airport_info["LocID"], airport_info["Hub\r\n(FY25)"]))

In [4]:
coordinates = pl.read_csv("./dat/T_MASTER_CORD.csv") # https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FLL&QO_fu146_anzr=N8vn6v10%20f722146%20gnoyr5
coordinates = coordinates[["AIRPORT", "DISPLAY_AIRPORT_CITY_NAME_FULL", "AIRPORT_STATE_CODE", "LATITUDE", "LONGITUDE"]]
coordinates = coordinates.with_columns(
    pl.col("AIRPORT").map_elements(lambda x: iata_to_local[x] if x in iata_to_local else x)
)
print(coordinates.shape)

code_to_name = dict(zip(coordinates["AIRPORT"], coordinates["DISPLAY_AIRPORT_CITY_NAME_FULL"]))
code_to_state = dict(zip(coordinates["AIRPORT"], coordinates["AIRPORT_STATE_CODE"]))
code_to_lat = dict(zip(coordinates["AIRPORT"], coordinates["LATITUDE"]))
code_to_lon = dict(zip(coordinates["AIRPORT"], coordinates["LONGITUDE"]))
print(len(coordinates))

(20188, 5)
20188


## DB1B

In [5]:
files = glob.glob("./dat/T_DB1B_COUPON_2024*.csv") # https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FLM&QO_fu146_anzr=b4vtv0%20n0q%20Qr56v0n6v10%20f748rB
print(files)

db1b = []
for file in files:
    tmp = pl.read_csv(file)
    db1b.append(tmp)
db1b = pl.concat(db1b)
db1b = db1b.filter(pl.col("YEAR")==2024)

['./dat\\T_DB1B_COUPON_2024Q1.csv', './dat\\T_DB1B_COUPON_2024Q2.csv', './dat\\T_DB1B_COUPON_2024Q3.csv', './dat\\T_DB1B_COUPON_2024Q4.csv']


In [6]:
# Standardizing airport codes to FAA
db1b = db1b.with_columns(
    pl.col("ORIGIN").replace_strict(iata_to_local),
    pl.col("DEST").replace_strict(iata_to_local)
)

In [7]:
elist = db1b.group_by(["ORIGIN", "DEST"]).agg(pl.col("PASSENGERS").sum())
N_T = len(set(elist["ORIGIN"]) | set(elist["DEST"]))
E_T = len(elist)
print(N_T, E_T)

456 17287


In [8]:
Airports = dispindiffs.DisparityInDifferences(elist, source="ORIGIN", target="DEST", weight="PASSENGERS")
Airports.calc_disp()
Airports.calc_disp_in_diffs()

Merging bilateral relations
Generating pre-sampled values from beta distributions
Calculating statistical significance
Done


### Disparity Filter

In [9]:
n_nodes_edges_by_th = []
for th in [10**(-k) for k in np.arange(20, -0.1, -0.25)]:
    bb, th, N, E = Airports.extr_disp_backbone(th=th)    
    n_nodes_edges_by_th.append((th, N, E))
pd.DataFrame(n_nodes_edges_by_th, columns=["th", "n_nodes", "n_edges"]).to_csv("./outputs/airports_disp_info_by_th.csv", index=False)

In [10]:
disp_backbone, _, _, _ = Airports.extr_disp_backbone(th=0.01)

### Disparity-in-Differences

In [11]:
n_nodes_edges_by_th = []
for th in [10**(-k) for k in np.arange(20, -0.1, -0.25)]:
    bb, th, N, E = Airports.extr_disp_in_diffs_backbone(th=th)    
    n_nodes_edges_by_th.append((th, N, E))
pd.DataFrame(n_nodes_edges_by_th, columns=["th", "n_nodes", "n_edges"]).to_csv("./outputs/airports_disp_in_diffs_info_by_th.csv", index=False)

In [12]:
disp_in_diffs_backbone, _, _, _  = Airports.extr_disp_in_diffs_backbone(th=0.01)    

### Validation

In [13]:
disp_backbone = disp_backbone.with_columns(
    pl.col("source").replace_strict(code_to_hub, default="").alias("source_hub"),
    pl.col("target").replace_strict(code_to_hub, default="").alias("target_hub"),
    pl.col("source").replace_strict(code_to_region, default="").alias("source_region"),
    pl.col("target").replace_strict(code_to_region, default="").alias("target_region")
).to_pandas()

disp_backbone["source_hub"] = pd.Categorical(disp_backbone["source_hub"], categories=["L", "M", "S", "N"])
disp_backbone["target_hub"] = pd.Categorical(disp_backbone["target_hub"], categories=["L", "M", "S", "N"])

disp_backbone = disp_backbone[disp_backbone["source_region"]==disp_backbone["target_region"]]

In [14]:
eval_disparity = []
for f in sorted(disp_backbone["source_region"].unique()):    
    bb_disparity_region = disp_backbone[disp_backbone["source_region"]==f].reset_index(drop=True)
    n = len(set(bb_disparity_region["source"]) | set(bb_disparity_region["target"]))
    mat = pd.crosstab(bb_disparity_region["source_hub"], bb_disparity_region["target_hub"])
    mat = mat.reindex(index=["L", "M", "S", "N"], columns=["L", "M", "S", "N"], fill_value=0)
    #print(mat)

    mat_np = np.array(mat)
    e = np.triu(mat_np, k=1).sum()/mat_np.sum()
    diag = np.diag(mat_np).sum()/mat_np.sum()

    eval_disparity.append((f, e, diag))

eval_disparity = pd.DataFrame(eval_disparity, columns=["region", "misaligned_rate", "diag_rate"])
eval_disparity["misaligned_rate"].mean(), eval_disparity["diag_rate"].mean()

(0.01961295253978181, 0.11096627684543975)

In [15]:
disp_in_diffs_backbone = disp_in_diffs_backbone.with_columns(
    pl.col("source").replace_strict(code_to_hub, default="").alias("source_hub"),
    pl.col("target").replace_strict(code_to_hub, default="").alias("target_hub"),
    pl.col("source").replace_strict(code_to_region, default="").alias("source_region"),
    pl.col("target").replace_strict(code_to_region, default="").alias("target_region")
).to_pandas()
disp_in_diffs_backbone["source_hub"] = pd.Categorical(disp_in_diffs_backbone["source_hub"], categories=["L", "M", "S", "N"])
disp_in_diffs_backbone["target_hub"] = pd.Categorical(disp_in_diffs_backbone["target_hub"], categories=["L", "M", "S", "N"])

disp_in_diffs_backbone = disp_in_diffs_backbone[disp_in_diffs_backbone["source_region"]==disp_in_diffs_backbone["target_region"]]
pd.crosstab(disp_in_diffs_backbone["source_hub"], disp_in_diffs_backbone["target_hub"])

target_hub,L,M,S,N
source_hub,,,,
L,6,0,0,0
M,44,3,0,0
S,115,11,4,0
N,184,13,1,13


In [16]:
eval_disp_in_diffs = []
for f in sorted(disp_in_diffs_backbone["source_region"].unique()):    
    bb_disparity_region = disp_in_diffs_backbone[disp_in_diffs_backbone["source_region"]==f].reset_index(drop=True)
    n = len(set(bb_disparity_region["source"]) | set(bb_disparity_region["target"]))
    mat = pd.crosstab(bb_disparity_region["source_hub"], bb_disparity_region["target_hub"])
    mat = mat.reindex(index=["L", "M", "S", "N"], columns=["L", "M", "S", "N"], fill_value=0)
    print(mat)

    mat_np = np.array(mat)
    e = np.triu(mat_np, k=1).sum()/mat_np.sum()
    diag = np.diag(mat_np).sum()/mat_np.sum()

    eval_disp_in_diffs.append((f, e, diag))
    
eval_disp_in_diffs = pd.DataFrame(eval_disp_in_diffs, columns=["region", "misaligned_rate", "diag_rate"])
eval_disp_in_diffs["misaligned_rate"].mean(), eval_disp_in_diffs["diag_rate"].mean()

target_hub  L  M  S  N
source_hub            
L           0  0  0  0
M           0  0  0  0
S           0  1  0  0
N           0  9  0  2
target_hub  L  M  S  N
source_hub            
L           0  0  0  0
M           0  0  0  0
S           0  1  0  0
N           0  0  0  0
target_hub  L  M  S  N
source_hub            
L           0  0  0  0
M           0  0  0  0
S           8  0  1  0
N           8  0  0  0
target_hub   L  M  S  N
source_hub             
L            0  0  0  0
M            4  1  0  0
S           14  0  0  0
N           47  0  0  1
target_hub  L  M  S  N
source_hub            
L           0  0  0  0
M           0  0  0  0
S           0  0  1  0
N           2  0  0  1
target_hub   L  M  S  N
source_hub             
L            1  0  0  0
M            5  0  0  0
S           15  0  0  0
N           47  0  0  3
target_hub   L  M  S  N
source_hub             
L            3  0  0  0
M           16  1  0  0
S           44  1  0  0
N           37  2  0  2
target_hub   L  

(0.0, 0.11247613455029873)

In [17]:
eval_agg = eval_disparity.merge(eval_disp_in_diffs, on="region")
eval_agg.columns = ["region", "misaligned_rate_disp", "diag_rate_disp", "misaligned_rate_disp_in_diffs", "diag_rate_disp_in_diffs"]
eval_agg["misaligned_rate_disp"] = eval_agg["misaligned_rate_disp"].round(3)
eval_agg["misaligned_rate_disp_in_diffs"] = eval_agg["misaligned_rate_disp_in_diffs"].round(3)
eval_agg.to_csv("./outputs/airports_eval_agg.csv", index=False)
eval_agg

,region,misaligned_rate_disp,diag_rate_disp,misaligned_rate_disp_in_diffs,diag_rate_disp_in_diffs
0,Alaskan,0.077,0.000000,0.0,0.166667
1,Central,0.000,0.333333,0.0,0.000000
2,Eastern,0.000,0.000000,0.0,0.058824
3,Great Lakes,0.000,0.014493,0.0,0.029851
4,New England,0.000,0.000000,0.0,0.500000
5,Northwest Mountain,0.000,0.081081,0.0,0.056338
6,Southern,0.016,0.146341,0.0,0.056604
7,Southwest,0.000,0.113924,0.0,0.101449
8,Western-Pacific,0.083,0.309524,0.0,0.042553
